In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import shutil
import pandas as pd

C:\Users\ramir\.conda\envs\525_2022\lib\site-packages\ipykernel\parentpoller.py:109: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  warnings.warn("""Parent poll failed.  If the frontend dies,


In [39]:
%cd /Users/ramir/OneDrive/Desktop/MDS_UBC_2021/labs_block6/525

C:\Users\ramir\OneDrive\Desktop\MDS_UBC_2021\labs_block6\525


In [2]:
# Downloading data 
article_id = 14096681 
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
#output
output_directory = "data/"
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]             
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
%%time

if os.path.exists(f"{output_directory}'data.zip'"):
    print("file exists")

else:
    files_to_dl = ["data.zip"]  
    for file in files:
        if file["name"] in files_to_dl:
            os.makedirs(output_directory, exist_ok=True)
            urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: total: 9.3 s
Wall time: 3min 17s


## Results:

| Team Member          | Operating System | RAM (GB) | Processor                 | Is SSD | Time taken |
| -------------------- | ---------------- | -------- | ------------------------- | ------ | ---------- |
|  Reza |  |  |  |  |  |
| Taiwo          |  |  |  |  |  |
| Allyson           |  |  |  |  |  |
| Ramiro          | Windows 10 x64   | 8     | Intel Core i5  | No    | 3 mins 17s |

In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: total: 25 s
Wall time: 2min 10s


## Results:

| Team Member          | Operating System | RAM (GB) | Processor                 | Is SSD | Time taken |
| -------------------- | ---------------- | -------- | ------------------------- | ------ | ---------- |
|  Reza Mirzazadeh |  |  |  |  |  |
| Taiwo          |  |  |  |  |  |
| Allyson Stoll           |  |  |  |  |  |
| Ramiro Mejia          | Windows 10 x64   | 8     | Intel Core i5  | No    | 2 mins 10s |

In [7]:
%ls figrainfall

 Volume in drive C is OS
 Volume Serial Number is 4AFF-EDFD

 Directory of C:\Users\ramir\OneDrive\Desktop\MDS_UBC_2021\labs_block6\525\figrainfall

03/28/2022  04:23 PM    <DIR>          .
03/28/2022  04:23 PM    <DIR>          ..
03/28/2022  02:39 PM    <DIR>          __MACOSX
03/28/2022  04:19 PM       127,613,760 ACCESS-CM2_daily_rainfall_NSW.csv
03/28/2022  04:20 PM       114,707,410 ACCESS-ESM1-5_daily_rainfall_NSW.csv
03/28/2022  04:19 PM        94,960,113 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
03/28/2022  04:20 PM       294,260,911 BCC-CSM2-MR_daily_rainfall_NSW.csv
03/28/2022  04:21 PM        55,224,437 BCC-ESM1_daily_rainfall_NSW.csv
03/28/2022  04:22 PM        46,286,371 CanESM5_daily_rainfall_NSW.csv
03/28/2022  04:20 PM       330,360,682 CMCC-CM2-HR4_daily_rainfall_NSW.csv
03/28/2022  04:21 PM       328,787,320 CMCC-CM2-SR5_daily_rainfall_NSW.csv
03/28/2022  04:20 PM       328,852,379 CMCC-ESM2_daily_rainfall_NSW.csv
03/28/2022  03:04 PM     6,782,678,136 combined_data.csv


In [8]:
# moving/excluding observed rain_fall to another folder 

observed_rain_fall = 'observed_daily_rainfall_SYD.csv'

os.makedirs(os.path.join(output_directory, 'observed_daily_rainfall_SYD'), exist_ok=True)

if not os.path.exists(os.path.join(output_directory, observed_rain_fall)):
    shutil.move(os.path.join(output_directory, file), os.path.join(output_directory, 'observed_daily_rainfall_SYD', file))

#### 1. Combine data CSVs into a single CSV using pandas.

In [9]:
%%time

files = glob.glob(f"{output_directory}/*.csv")
                  
df = pd.concat((pd.read_csv(file, index_col=0, parse_dates=True)
                .assign(file=re.findall(r'(?<=\\)(.*)(?=_daily)', file)[0])
                for file in files)
              )

df.to_csv(os.path.join(output_directory, "combined_data.csv"))

CPU times: total: 9min 19s
Wall time: 10min 26s


In [12]:
%%time
data = pd.read_csv(os.path.join(output_directory, "combined_data.csv") ,index_col=0, parse_dates=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62513863 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float64
 1   lat_max        float64
 2   lon_min        float64
 3   lon_max        float64
 4   rain (mm/day)  float64
 5   file           object 
dtypes: float64(5), object(1)
memory usage: 3.3+ GB
CPU times: total: 2min 25s
Wall time: 8min 58s


## Results:

| Team Member          | Operating System | RAM (GB) | Processor                 | Is SSD | Time taken |
| -------------------- | ---------------- | -------- | ------------------------- | ------ | ---------- |
|  Reza Mirzazadeh |  |  |  |  |  |
| Taiwo          |  |  |  |  |  |
| Allyson Stoll           |  |  |  |  |  |
| Ramiro Mejia          | Windows 10 x64   | 8     | Intel Core i5  | No    | 2 mins 25s |

## Changing dtypes

In [15]:
%%time

types = {'lat_min': 'float16', 
          'lat_max': 'float16', 
          'lon_min': 'float16', 
          'lon_max': 'float16',
          'rain (mm/day)':'float16',
          'file':'str'}

df = pd.read_csv(os.path.join(output_directory, "combined_data.csv"),index_col=0, parse_dates=True, dtype=types)

CPU times: total: 2min 6s
Wall time: 5min 5s


## Results:

| Team Member          | Operating System | RAM (GB) | Processor                 | Is SSD | Time taken |
| -------------------- | ---------------- | -------- | ------------------------- | ------ | ---------- |
|  Reza Mirzazadeh |  |  |  |  |  |
| Taiwo          |  |  |  |  |  |
| Allyson Stoll           |  |  |  |  |  |
| Ramiro Mejia          | Windows 10 x64   | 8     | Intel Core i5  | No    | 2 mins 6s |

## Selected columns

In [17]:
%%time

cols = ["rain (mm/day)","file"]
types = {'rain (mm/day)':'float16',
        'file':'str'}
df = pd.read_csv(os.path.join(output_directory, "combined_data.csv"),
                 index_col=0, parse_dates=True, usecols=cols, dtype=types)

CPU times: total: 2min 7s
Wall time: 4min 16s


## Results:

| Team Member          | Operating System | RAM (GB) | Processor                 | Is SSD | Time taken |
| -------------------- | ---------------- | -------- | ------------------------- | ------ | ---------- |
|  Reza Mirzazadeh |  |  |  |  |  |
| Taiwo          |  |  |  |  |  |
| Allyson Stoll           |  |  |  |  |  |
| Ramiro Mejia          | Windows 10 x64   | 8     | Intel Core i5  | No    | 2 mins 7s |

## Chunks

In [19]:
%%time

counts = pd.Series(dtype=int)
for chunk in pd.read_csv(os.path.join(output_directory, "combined_data.csv"), chunksize=100_000):
    counts = counts.add(chunk["file"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
observed              46020
dtype: int32
CPU times: total: 1min 40s
Wall time: 3min 49s


## Results:

| Team Member          | Operating System | RAM (GB) | Processor                 | Is SSD | Time taken |
| -------------------- | ---------------- | -------- | ------------------------- | ------ | ---------- |
|  Reza Mirzazadeh |  |  |  |  |  |
| Taiwo          |  |  |  |  |  |
| Allyson Stoll           |  |  |  |  |  |
| Ramiro Mejia          | Windows 10 x64   | 8     | Intel Core i5  | No    | 1 mins 40s |

## EDA